In [ ]:
!pip install pillow faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.0 MB/s eta 0:00:00


In [1]:
from PIL import Image, ImageDraw, ImageFont
import random
import textwrap
import os
from IPython.display import display
from torchvision.transforms import v2 as tvt

In [16]:
from typing import ItemsView
def generate_receipt(store_name, items, total, output_path):
    width, height = 400, 600
    image = Image.new("RGB", (width, height), (250,250,250))
    draw = ImageDraw.Draw(image)

    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except:
        font = ImageFont.load_default()

    y = 20
    draw.text((20, y), store_name, font=font, fill="black")
    y += 30
    draw.text((20, y), "--------------------------", font=font, fill="black")
    y += 20

    for item, price in items:
        wrapped_text = textwrap.wrap(item, width=25)
        for line in wrapped_text:
            draw.text((20, y), line, font=font, fill="black")
            y += 20
        draw.text((300, y - 20), f"${price:.2f}", font=font, fill="black")

    y += 20
    draw.text((20, y), "--------------------------", font=font, fill="black")
    y += 20
    draw.text((20, y), f"Total: ${total:.2f}", font=font, fill="black")

    image = augment_img(image,1)

    image.save(output_path)
    return image

In [15]:
def create_random_receipt(output_folder="receipts", number=0):
    businesses = {
        "Retail Stores": ["Walmart", "Target", "Best Buy", "IKEA", "GameStop"],
        "Grocery Stores": ["Costco", "Trader Joe's", "Whole Foods", "Loblaws"],
        "Fast Food Chains": ["McDonald's", "Burger King", "Subway", "KFC"],
        "Restaurants": ["Olive Garden", "Red Lobster", "Applebee's"],
        "Bars & Cafés": ["Starbucks", "Tim Hortons", "Dunkin’ Donuts"],
        "Cinemas": ["Cineplex", "AMC Theatres", "Regal Cinemas"],
        "Gas Stations": ["Shell", "Esso", "Chevron"],
        "Tech Stores": ["Apple Store", "Microsoft Store", "Samsung Store"],
        "Pharmacies": ["CVS", "Walgreens", "Shoppers Drug Mart"],
        "Hotels & Travel": ["Marriott", "Hilton", "Uber", "Lyft", "Airbnb"],
        "Pet Stores": ["PetSmart", "Petco"],
    }

    item_catalog = {
        "Retail Stores": ["Laptop", "Smartphone", "Headphones", "Gaming Console"],
        "Grocery Stores": ["Milk", "Bread", "Cheese", "Chicken Breast"],
        "Fast Food Chains": ["Big Mac", "Fries", "Soft Drink", "Chicken Nuggets"],
        "Restaurants": ["Steak Dinner", "Pasta Alfredo", "Caesar Salad"],
        "Bars & Cafés": ["Latte", "Cappuccino", "Beer Pint", "Muffin"],
        "Cinemas": ["Movie Ticket", "Popcorn", "Soft Drink", "Nachos"],
        "Gas Stations": ["Gasoline (Liters)", "Car Wash", "Motor Oil"],
        "Tech Stores": ["MacBook", "iPhone", "Wireless Mouse"],
        "Pharmacies": ["Pain Reliever", "Cough Syrup", "Vitamins"],
        "Hotels & Travel": ["Hotel Stay", "Taxi Ride", "Baggage Fee"],
        "Pet Stores": ["Dog Food", "Cat Litter", "Pet Shampoo"],
    }

    category = random.choice(list(businesses.keys()))
    store_name = random.choice(businesses[category])
    items = random.choices(item_catalog[category], k=random.randint(2, 20))

    item_prices = [(item, round(random.uniform(3, 150), 2)) for item in items]
    total = sum(price for _, price in item_prices)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    receipt_name = f"{str(number).replace(' ', '_')}_receipt.png"
    output_path = os.path.join(output_folder, receipt_name)
    return generate_receipt(store_name, item_prices, total, output_path), len(item_prices)

In [4]:
def augment_img(img,p):
  trans1 = tvt.RandomApply(transforms=[tvt.ElasticTransform(alpha=random.randint(20,25)),
                                    tvt.GaussianBlur(kernel_size=(5,9),sigma=(0.1,1)),
                                    ], p=p)
  trans2 = tvt.RandomPerspective(distortion_scale=0.3,p=1,fill=random.randint(0,255))
  return trans2(trans1(img))

In [20]:
#### Generate Receipt dataset of any size
num_receipts = 10
total_items = 0
for i in range(num_receipts):
  receipt_image, num_items = create_random_receipt(number=i)
  total_items += num_items
print(total_items)

138
